In [ ]:
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd
import threading

topics = ['thoi-su','the-gioi','kinh-te','doi-song','suc-khoe','gioi-tre','giao-duc','van-hoa','giai-tri','the-thao','cong-nghe-game','xe','du-lich']
cac_danh_muc = ['Thời sự',
                'Thế giới',
                'Kinh tế',
                'Đời sống',
                'Sức khỏe',
                'Giới trẻ',
                'Giáo dục',
                'Văn hóa',
                'Giải trí',
                'Thể thao',
                'Công nghệ - Game',
                'Xe',
                'Du lịch',]
base_url = 'https://thanhnien.vn'

article_links = []
data = []
lock = threading.Lock()

In [8]:
def get_link(url):
    print(len(article_links))
    #Kiểm tra nếu lỗi thì bỏ qua
    try:
        response = rq.get(f"{base_url}{url}")
        if response.status_code == 200:
            soup = BeautifulSoup(response.content,"html.parser")
            #Tìm all các ô có đường dẫn
            content = soup.find_all('a',class_='box-category-link-title')
            # Lấy link bài viết ở trang
            for article in content:
                with lock:
                    #Kiểm tra href này có trong article_links chưa nếu chưa thì thêm vào
                    if article['href'] not in article_links:
                        article_links.append(article['href'])
                        # print(article['href'])
        else:
            print("Không thể truy cập trang web.")
    except:
        return
#Lặp qua các topic lấy các href
for topic in topics:
    get_link("/"+topic)
#Dùng đa luồng lặp lấy các href từ article_links
threads = []
# max_threads = 10000
for link in article_links:
    #Nếu số lượng link lớn hơn 30000 dừng
    if len(article_links) > 30000:
        break
    thread = threading.Thread(target=get_link, args=(link,))
    thread.start()
    threads.append(thread)

<Thread(Thread-1410 (get_link), started 17752)>
<Thread(Thread-1411 (get_link), started 20460)>
<Thread(Thread-1412 (get_link), started 2516)>
<Thread(Thread-1413 (get_link), started 20296)>
<Thread(Thread-1414 (get_link), started 11408)>
<Thread(Thread-1415 (get_link), started 11816)>
<Thread(Thread-1416 (get_link), started 4684)>
<Thread(Thread-1417 (get_link), started 4884)>
<Thread(Thread-1418 (get_link), started 19580)>
<Thread(Thread-1419 (get_link), started 4300)>
<Thread(Thread-1420 (get_link), started 6252)>
<Thread(Thread-1421 (get_link), started 148)>
<Thread(Thread-1422 (get_link), started 19704)>
<Thread(Thread-1423 (get_link), started 7864)>
<Thread(Thread-1424 (get_link), started 18684)>
<Thread(Thread-1425 (get_link), started 16676)>
<Thread(Thread-1426 (get_link), started 3852)>
<Thread(Thread-1427 (get_link), started 14316)>
<Thread(Thread-1428 (get_link), started 11136)>
<Thread(Thread-1429 (get_link), started 15556)>
<Thread(Thread-1430 (get_link), started 17416)>
<

In [13]:
print(len(article_links))
print(len(set(article_links)))

10040
10040


In [ ]:
def get_content(url):
    print(len(data))
    try:
        html_sub = rq.get(base_url+url)
        # Use BeautifulSoup to parse the HTML
        soup_sub = BeautifulSoup(html_sub.content, 'html.parser')
        if soup_sub.find('div',class_ = 'detail-cate'):
        #Lấy danh mục
            danh_muc = soup_sub.find('div',class_ = 'detail-cate').find('a').text.strip()
            if danh_muc not in cac_danh_muc:
                return
            #Lấy nội dung của bài viết
            noi_dung = noi_dung = soup_sub.find('div',class_='detail-cmain').get_text().replace('\n','')
            #Lưu danh mục và nội dung dưới dạng từ điển
            with lock:
                data.append({'Danh mục':danh_muc,'Nội dung':noi_dung})
        else:
            return
    except:
        return
#Dùng đa luồng lấy danh mục và nội dung từ các link đã crawl
threads = []
for link in article_links:
    thread = threading.Thread(target=get_content, args=(link,))
    thread.start()
    threads.append(thread)

In [ ]:
print(len(data))

In [ ]:
# Lưu data thành file csv
df = pd.DataFrame(data)
df.to_csv('Data.csv', index=False)